<a href="https://colab.research.google.com/github/jm-h98/easy_DG_BS/blob/main/easy_DG_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to easily use BigGAN:
1. Install the requirements
2. Run either deep gaze OR big sleep, you cant run both at the same time!
3. Create a .zip-file of the generated images
4. Create a video of the progress

In [ ]:
# test gpu: K80 < P4 < T4 < P100
!nvidia-smi

In [ ]:
# install requirements
!pip install deep-daze --upgrade
!pip install big-sleep --upgrade

In [ ]:
# deep daze https://github.com/lucidrains/deep-daze
# using openai's CLIP and Siren to guide BigGAN
# if Upload_Image is activated, upload your image using the button below
# produces more abstract images

# you can only run this or the next code field at a time

from tqdm.notebook import trange
from IPython.display import Image, display
from PIL import Image as i

from deep_daze import Imagine
import glob
import os

imgs = glob.glob("/content/*.png")
for img in imgs:
  try:
    os.remove(img)
  except:
    pass

TEXT = '' #@param {type:"string"}
DISPLAY_EVERY =  10 #@param {type:"number"}
IMAGE_WIDTH = 512 #@param {type:"number"}
Upload_Image = False #@param {type:"boolean"}
EPOCHS = 20
img_path = None
ITERATIONS = 500
if Upload_Image:
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    img_path = fn
  IMG = i.open(img_path)
else:
  IMG = None
model = Imagine(
    img = IMG,
    epochs = EPOCHS,
    text = TEXT,
    num_layers = 32,
    save_every = DISPLAY_EVERY,
    image_width = IMAGE_WIDTH,
    lr = 1e-5,
    iterations = ITERATIONS,
    save_progress = True
)
for epoch in trange(EPOCHS, desc = 'epochs'):
    for i in trange(ITERATIONS, desc = 'iteration'):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        filename = TEXT.replace(' ', '_')
        image = Image(f'./{filename}.jpg')
        display(image)

In [ ]:
# big sleep https://github.com/htoyryla/big-sleep
# using openai's CLIP to guide BigGAN
# produces more realistic images

# you can only run this or the previous code field at a time

from tqdm.notebook import trange
from IPython.display import Image, display
from big_sleep import Imagine
import glob
import os

imgs = glob.glob("/content/*.png")
for img in imgs:
  try:
    os.remove(img)
  except:
    pass

TEXT = '' #@param {type:"string"}
SAVE_EVERY = 20 #@param {type:"number"}
SAVE_PROGRESS = True
LEARNING_RATE = 5e-2
ITERATIONS = 200
SEED = 123456789 #@param {type: "number"}
EPOCHS = 20
model = Imagine(
    text = TEXT,
    save_every = SAVE_EVERY,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS,
    seed = SEED)
for epoch in trange(EPOCHS, desc = 'epochs'):
    for i in trange(ITERATIONS, desc = 'iteration'):
        model.train_step(epoch, i)
        if i == 0 or i % model.save_every != 0:
            continue
        filename = TEXT.replace(' ', '_')
        image = Image(f'./{filename}.png')
        display(image)

# Download images or generate video
1. Run the image renamer
2. (optional) Download .zip-file
3. (optional) Generate and download .mp4-file

In [5]:
# rename the images

import glob
import os

imgs = glob.glob("/content/*.png")
for img in imgs:
  try:
    name, num = img[:-4].split('.')
    new_name = '{}.{}.png'.format(name, num.zfill(5))
    os.rename(img, new_name)
  except:
    pass

In [ ]:
# download everything from /contents/ as a .zip-file
# download can take quite some time

!zip -r /content/file.zip /content

from google.colab import files
files.download("/content/files.zip")

In [ ]:
# generate video and download it

import cv2
import numpy as np

FPS = 60 #@param {type:"integer"}

imgs = glob.glob("/content/*.png")
imgs.extend(glob.glob("/content/*.jpg"))
imgs = sorted(imgs)
i = cv2.imread(imgs[0])
height, width, _ = i.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter('/content/video.mp4', fourcc, FPS, (width,height))

for img in imgs:
  img = cv2.imread(img)
  video.write(img)

video.release()
cv2.destroyAllWindows()

from google.colab import files
files.download("/content/video.mp4")